# CLUSTERING the neighborhood of toranto

### capston week3 project

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

##### importing the required packages

In [2]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source,'lxml')
table_soup=soup.table.tbody.find_all('tr')
table_soup.remove(table_soup[0])
pc=list()
br=list()
ne=list()
for row in table_soup:
    try:
        pc.append(row.find_all('td')[0].text)
        br.append(row.find_all('td')[1].text)
        ne.append(row.find_all('td')[2].text.strip())
    except Exception as e:
        pass
toranto_df=pd.DataFrame({'Postalcode':pc,'Borough':br,'Neighborhood':ne})

- geting the html page from the web and striping the table information into the arrays
- and storing the arrays data into data frame 

In [3]:
toranto_df=toranto_df.drop(toranto_df[toranto_df['Borough']=='Not assigned'].index)

##### remving the cells which are having 'Not assigned' in "Borough"

In [4]:
toranto_df.loc[toranto_df['Neighborhood']=='Not assigned','Neighborhood']=toranto_df.loc[toranto_df['Neighborhood']=='Not assigned','Borough']

grouping data by postal code

In [5]:
toranto_gr=toranto_df.groupby("Postalcode")

In [6]:
toranto_gr

#### joining the Neighborhoods acc. to postalcode

In [7]:
df=pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'])
i=0
for grp in pd.unique(toranto_df["Postalcode"]):
    ni=list()
    grp_df=toranto_gr.get_group(grp)
    ni=grp_df['Neighborhood'].values
    #temp_df=pd.DataFrame({'Postalcode':[grp],'Borough':pd.unique(grp_df['Borough']),'Neighborhood':','.join(ni)})
    #print(temp_df)
    df.loc[i]=[grp,pd.unique(grp_df['Borough'])[0],','.join(ni)]
    i=i+1
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
pip install geocoder # import geocoder

Note: you may need to restart the kernel to use updated packages.


In [8]:
!wget -o 'toranto_geodata.csv' https://cocl.us/Geospatial_data
data=pd.read_csv('https://cocl.us/Geospatial_data')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
lat_arr=list()
lon_arr=list()
for postal_code in df['Postalcode'].values:
    lat_arr.append(data.loc[data['Postal Code']==postal_code,'Latitude'].values[0])
    lon_arr.append(data.loc[data['Postal Code']==postal_code,'Longitude'].values[0])
df['Latitude']=lat_arr
df['Longitude']=lon_arr
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### geting longitude and latitude from _.csv_ file

In [12]:
print('SHAPE :',df.shape)

SHAPE : (103, 5)


##### imorting packeges neede for the maps

In [13]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

### generating a map around toronto visulizing the borough aroud it

In [14]:
map_toranto = folium.Map(location=[43.6532,-79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toranto)  
    
map_toranto

In [15]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [16]:
CLIENT_ID = '0WXUPARWL4NOWJIKSZGOSK0JNATJBA5ITT3ECJ1M3T3CL5QZ' # your Foursquare ID
CLIENT_SECRET = 'ZZYXGDMG23IK2XJZQ0YED33TX3E2NHEQAFVE5M2DLWTFE5DY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0WXUPARWL4NOWJIKSZGOSK0JNATJBA5ITT3ECJ1M3T3CL5QZ
CLIENT_SECRET:ZZYXGDMG23IK2XJZQ0YED33TX3E2NHEQAFVE5M2DLWTFE5DY


In [17]:
print(df['Neighborhood'].values[0],df['Postalcode'].values[0])

Parkwoods M3A


In [18]:
print('latitude :',df['Latitude'].values[0])
print('longitude:',df['Longitude'].values[0])

latitude : 43.7532586
longitude: -79.3296565


### analizing the neighborhood of 'parkwoods' under postal code 'M3A'  with limits under 2 kilometers

In [19]:
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
 CLIENT_SECRET, 
    VERSION, 
    43.7532586,
-79.3296565, 
    2000,#two kilometer 
    100)

 Assuming that the customer wanted to start new business in 'parkwoods'.

In [20]:
results = requests.get(url).json()
print('there are {} famous venues near by'.format(results['response']['totalResults']))

there are 93 famous venues near by


### Vsisulazing the venues nearby

In [29]:
res2=results['response']['groups'][0]['items']
for res in res2:
    print(res['venue']['name'])
    print("     at a distance of---> ",res['venue']['location']['distance'])
    print()
print()

Allwyn's Bakery
     at a distance of--->  833

Donalda Golf & Country Club
     at a distance of--->  1053

Brookbanks Park
     at a distance of--->  245

Graydon Hall Manor
     at a distance of--->  1597

Darband Restaurant
     at a distance of--->  1530

LA Fitness
     at a distance of--->  1532

Island Foods
     at a distance of--->  1553

Tim Hortons
     at a distance of--->  866

Galleria Supermarket
     at a distance of--->  1597

LCBO
     at a distance of--->  1336

Me Va Me Kitchen Express
     at a distance of--->  1798

North Beach Indoor Volleyball Academy
     at a distance of--->  1851

Matsuda Japanese Cuisine & Teppanyaki
     at a distance of--->  1561

A&W Canada
     at a distance of--->  852

Gilaneh
     at a distance of--->  1639

The Captain's Boil
     at a distance of--->  1609

Tim Hortons
     at a distance of--->  1254

Spoon And Fork Plus
     at a distance of--->  1614

Baretto Caffé
     at a distance of--->  1669

Shooters Snooker & Sports Club
 

In [25]:
map_parkwoods=folium.Map(location=[43.7532586,-79.3296565],zoom_start=13)
map_parkwoods

In [26]:
for res in res2:
    latit=res['venue']['location']['lat']
    longit=res['venue']['location']['lng']
    label=res['venue']['name']
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latit, longit],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_parkwoods)
map_parkwoods

In [32]:
df_clustered=pd.DataFrame(columns=['venue','latitude','longitude','venue_type'])
for res in res2:
    latit=res['venue']['location']['lat']
    longit=res['venue']['location']['lng']
    label=res['venue']['name']
    name=res['venue']['categories'][0]['name']
    df_clustered=df_clustered.append({'venue':label,'latitude':latit,'longitude':longit,'venue_type':name},ignore_index=True)
df_clustered.head(9)

,venue,latitude,longitude,venue_type
0,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Brookbanks Park,43.751976,-79.332140,Park
3,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,Darband Restaurant,43.755194,-79.348498,Middle Eastern Restaurant
5,LA Fitness,43.747665,-79.347077,Gym / Fitness Center
6,Island Foods,43.745866,-79.346035,Caribbean Restaurant
7,Tim Hortons,43.760668,-79.326368,Café
8,Galleria Supermarket,43.753520,-79.349518,Supermarket


### clustering and visualizing

In [33]:
from sklearn.cluster import KMeans
print('import succesful')

import succesful


In [34]:
kclusters=6
kmeans = KMeans(n_clusters=kclusters)
kmeans.fit(df_clustered[['latitude','longitude']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [35]:
df_clustered['cluster']=kmeans.labels_

In [36]:
df_clustered.head(9)

,venue,latitude,longitude,venue_type,cluster
0,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant,3
1,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course,0
2,Brookbanks Park,43.751976,-79.332140,Park,3
3,Graydon Hall Manor,43.763923,-79.342961,Event Space,0
4,Darband Restaurant,43.755194,-79.348498,Middle Eastern Restaurant,0
5,LA Fitness,43.747665,-79.347077,Gym / Fitness Center,5
6,Island Foods,43.745866,-79.346035,Caribbean Restaurant,5
7,Tim Hortons,43.760668,-79.326368,Café,3
8,Galleria Supermarket,43.753520,-79.349518,Supermarket,0


In [37]:
import matplotlib.cm as cm
import matplotlib.colors as colors


In [38]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_clusters=folium.Map(location=[43.7532586,-79.3296565],zoom_start=13)

markers_colors = []
for lat, lon, poi, cluster in zip(df_clustered['latitude'], df_clustered['longitude'], df_clustered['venue'], df_clustered['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

 ### modeled by: K.lokanadh reddy 
    

 #### lets assume that the business customer is gonig to start is a **supermarket**

,cluster,count,btype_count


In [70]:
cluster=df_clustered['cluster'].value_counts().index
count=df_clustered['cluster'].value_counts().values
btype_count=list()
for i in cluster:
    btype_count.append(sum(1 for p in ((df_clustered[df_clustered['cluster']==i]['venue_type']=='Supermarket').values) if p==True))
df_clustered_btype=pd.DataFrame({'cluster':cluster,'count':count,'btype_count':btype_count})
df_clustered_btype
    

,cluster,count,btype_count
0,0,32,1
1,1,18,0
2,3,16,1
3,4,11,0
4,2,10,0
5,5,6,0


 we can see that there are only two shopping malls one in **cluster 0 _and other in_ cluster 3**
 
 lets visualize that

In [90]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_clusters=folium.Map(location=[43.7532586,-79.3296565],zoom_start=14)

markers_colors = []
for lat, lon, poi, cluster,venue_type in zip(df_clustered['latitude'], df_clustered['longitude'], df_clustered['venue'], df_clustered['cluster'],df_clustered['venue_type']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if venue_type=='Supermarket':
        folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.9).add_to(map_clusters)
    else:
        folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.4).add_to(map_clusters)
       
map_clusters

**we can see the locations of the existing shopping malls in "BLACK" .**

#####  now lets filter the clusters which are having very few 'Shopping malls'(in this case 0)

In [83]:
df_clustered_btype_fil=df_clustered_btype[df_clustered_btype['btype_count']==df_clustered_btype['btype_count'].min()]
df_clustered_btype_fil

,cluster,count,btype_count
1,1,18,0
3,4,11,0
4,2,10,0
5,5,6,0


##### now lets find the clusters with high business chances( it is with more venues around)

In [84]:
df_clustered_btype_fil=df_clustered_btype_fil[df_clustered_btype_fil['count']==df_clustered_btype_fil['count'].max()]
df_clustered_btype_fil

,cluster,count,btype_count
1,1,18,0


### so, we can conclude that the best suitable cluster in "parkwoods" is cluster 1

In [87]:
kmeans.cluster_centers_[1]

array([ 43.75736004, -79.31265883])

## thank you

### model by : k .lokanadh reddy